# Introdução

*Script* que cria uma tabela que apresenta informações das Agências Ambientais da CETESB que atuam nos 645 município deo Estado de São Paulo.

In [ ]:
import os
import time
import requests
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup

## Função

In [ ]:
def get_AgenciaAmbiental(mun):
    # Município
    mun_url = mun.encode('ISO-8859-1')
    mun_url = urllib.parse.quote(mun_url)
    print('{} > {}'.format(mun, mun_url))
    
    # Set URL
    url = 'https://licenciamento.cetesb.sp.gov.br/municipioss.asp?muni={}'.format(mun_url)
    
    # Get Data
    r = requests.get(url)    
    soup = BeautifulSoup(r.content, 'html.parser')
    time.sleep(1)

    # Get all relevant data
    table = soup.find('table',  attrs={'width':'300'})
    rows = table.find_all('tr')
    
    # Create List Adress
    list_end = [mun]
    for row in rows:
        text = row.get_text()
        text = ' '.join(text.split())
        text = text.strip()
        list_end.append(text)
    
    # Rename List    
    for i in ['Bairro:', 'Cidade:', 'CEP:', 'Fone:', 'Fax:', 'E-mail:']:
        list_end = np.char.replace(list_end, i, '')
    list_end = [x.strip() for x in list_end]
    
    # Format Variables
    cep = '{:08d}'.format(int(list_end[5]))
    tel = list_end[6].strip().replace(' ', '').replace('-', ' ')
    fax = list_end[7].strip().replace(' ', '').replace('-', ' ')
    
    # Create Dictionary
    return {
        'municipio_cetesb' : list_end[0],
        'agencia'          : list_end[1],
        'endereco'         : list_end[2].split('nº')[0].strip(),
        'numero'           : list_end[2].split('nº')[1].strip(),
        'bairro'           : list_end[3],
        'municipio_sede'   : list_end[4].title(),
        'cep'              : '{}-{}'.format(cep[:5], cep[5:8]),
        'telefone'         : '({}) {}-{}'.format(tel[:2], tel[3:7], tel[7:]),
        'fax'              : '({}) {}-{}'.format(fax[:2], fax[3:7], fax[7:]),
        'email'            : list_end[8],
        'url'              : url
    }

Uma vez que a função foi definida, basta fazer um teste. Escolhi o município de Piracicaba, pois não tem acentos e grafias divergentes nas diferentes bases de dados.

In [ ]:
get_AgenciaAmbiental('Piracicaba')

## *Loop*

Há municípios que apresentam apóstrofo e muitas vezes é escrito como acento ou até mesmo divergência na grafia do nome do município, por exemplo, "Moji Mirim" e "Mogi Mirim". Por isso, optei por ler uma tabela que tem os nomes dos municípios utilizados pela CETESB e, uma vez que fiz isso, basta inserir o nome que a instituição padronizou, por meio de um *loop* (*for*).

In [ ]:
# Read Dataframe
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_municipios.csv'),
)

# Results
df.head()

In [ ]:
# Create empty list
list_addresses = []

#df = df[0:4]  # For tests only
for index, row in df.iterrows():
    # Create Small Dictionary
    dict_df = {
        'id_municipio': row['id_municipio'],        
    }
    
    # Get Addresses
    list_address = get_AgenciaAmbiental(row['municipio_cetesb'])
    list_address = {**dict_df, **list_address}
    list_addresses.append(list_address)

## *Create Table*

In [ ]:
df_addresses = pd.DataFrame.from_dict(list_addresses)

# Results
df_addresses.head()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
)
df_mun = df_mun[['id_municipio', 'municipio_nome']]

df_addresses = pd.merge(
    df_mun,
    df_addresses,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

df_addresses

In [ ]:
df_addresses.to_csv(
    os.path.join('data', 'tabs', 'tab_municipios_cetesb.csv'),
    index=False,
)

In [ ]:
# Create copy
df = df_addresses

# Get column's name
columns = list(df.columns[3:-1])
columns

# Group
df = pd.DataFrame(df.groupby(columns)['id_municipio'].count())
df = df.reset_index()
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

# Save
df.to_csv(
    os.path.join('data', 'tabs', 'tab_cetesb.csv'),
    index=False,    
)

# Results
df.head()

___

# Encerramento

## *Requirements*

In [ ]:
# Input
inp = 'get_infos.ipynb'

# Output
out = os.path.join(inp.split('.')[0])
!jupyter-nbconvert $inp --to python --output $out

print('-'*80)
!pipreqs '.' --force --debug

print('-'*80)
os.remove('{}.py'.format(out))
print('Arquivo {}.py deletado com sucesso!'.format(out))

## *GitHub*

In [ ]:
!nbstripout --install --attributes .gitattributes
!git status
!git add .
!git commit -m 'Arrumações Gerais'
!git push origin main